# STEP 1: Analyze Real Cities (500×500m)
## Extract Urban Metrics & Building Footprint Library

**Goal**: Analyze three 500×500m urban areas to extract:
- Space syntax metrics (nodes, edges, districts, landmarks)
- Building footprint library (individual building shapes)
- Building parcels (land use boundaries)
- Building geometry distributions

**Cities**:
1. Hanoi, Vietnam (21.0230°N, 105.8560°E) - Dense, organic layout
2. Brussels, Belgium (50.8477°N, 4.3572°E) - European historic core
3. Marrakech, Morocco (31.623811°N, -7.988662°W) - Compact medina

**Outputs**:
- GeoJSON files (nodes, edges, buildings, parcels, districts)
- JSON metrics file (urban_metrics.json)
- Building footprint library (building_footprint_library.json)
- Visualizations (PNG + SVG)
- Metrics summary table (CSV)

## 1. Setup & Configuration

In [ ]:
# Imports
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib import cm
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, box
from shapely.ops import unary_union
from shapely.affinity import translate
import json
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Configure OSMnx
ox.settings.use_cache = True
ox.settings.log_console = False

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Libraries imported successfully")

In [ ]:
# Configuration
CITIES = {
    'hanoi': {
        'name': 'Hanoi, Vietnam',
        'coords': (21.0230, 105.8560),
        'color': '#FF6B6B'
    },
    'brussels': {
        'name': 'Brussels, Belgium',
        'coords': (50.8477, 4.3572),
        'color': '#4ECDC4'
    },
    'marrakech': {
        'name': 'Marrakech, Morocco',
        'coords': (31.623811, -7.988662),
        'color': '#FFE66D'
    }
}

# Analysis parameters (adapted for 500×500m)
RADIUS = 250  # meters
REACH_RADII = [200, 300]
LOCAL_LANDMARK_RADIUS = 300
MIN_PARCEL_AREA = 500  # m²
MAX_PARCEL_AREA = 10000  # m²
FOOTPRINTS_PER_CITY = 35  # Target library size

# Output paths
OUTPUT_DIR = Path('outputs')
GEOJSON_DIR = OUTPUT_DIR / 'geojson'
VIZ_PNG_DIR = OUTPUT_DIR / 'visualizations' / 'png'
VIZ_SVG_DIR = OUTPUT_DIR / 'visualizations' / 'svg'
METRICS_DIR = OUTPUT_DIR / 'metrics'

for d in [GEOJSON_DIR, VIZ_PNG_DIR, VIZ_SVG_DIR, METRICS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("✓ Configuration complete")
print(f"  Analyzing {len(CITIES)} cities")
print(f"  Coverage radius: {RADIUS}m (~{RADIUS*2}×{RADIUS*2}m)")
print(f"  Output: {OUTPUT_DIR.absolute()}")

## 2. Data Acquisition

In [ ]:
# Download data for all cities
city_data = {}

for city_key, city_info in CITIES.items():
    print(f"\n{'='*60}")
    print(f"Downloading: {city_info['name']}")
    print(f"{'='*60}")
    
    lat, lon = city_info['coords']
    
    try:
        # 1. Street network
        print(f"  → Street network...")
        G = ox.graph_from_point((lat, lon), dist=RADIUS, network_type='walk', simplify=True)
        G_proj = ox.project_graph(G)
        
        # 2. Buildings
        print(f"  → Buildings...")
        buildings = ox.features_from_point((lat, lon), dist=RADIUS, tags={'building': True})
        buildings_proj = buildings.to_crs(ox.graph_to_gdfs(G_proj, nodes=False).crs)
        buildings_proj = buildings_proj[buildings_proj.geometry.type.isin(['Polygon', 'MultiPolygon'])].copy()
        
        # Convert MultiPolygons to Polygons
        def get_polygon(geom):
            if geom.geom_type == 'Polygon':
                return geom
            elif geom.geom_type == 'MultiPolygon':
                return max(geom.geoms, key=lambda p: p.area)
            return geom
        
        buildings_proj['geometry'] = buildings_proj.geometry.apply(get_polygon)
        buildings_proj = buildings_proj[buildings_proj.geometry.type == 'Polygon'].copy()
        
        # 3. Building Parcels (landuse)
        print(f"  → Building parcels (landuse)...")
        try:
            parcels = ox.features_from_point(
                (lat, lon),
                dist=RADIUS,
                tags={'landuse': True}
            )
            parcels_proj = parcels.to_crs(ox.graph_to_gdfs(G_proj, nodes=False).crs)
            parcels_proj = parcels_proj[parcels_proj.geometry.type.isin(['Polygon', 'MultiPolygon'])].copy()
            parcels_proj['geometry'] = parcels_proj.geometry.apply(get_polygon)
            parcels_proj = parcels_proj[parcels_proj.geometry.type == 'Polygon'].copy()
            print(f"    ✓ Found {len(parcels_proj)} parcels")
        except Exception as e:
            print(f"    ⚠ No parcels found: {e}")
            parcels_proj = gpd.GeoDataFrame(columns=['geometry', 'landuse'], crs=ox.graph_to_gdfs(G_proj, nodes=False).crs)
        
        # Store data
        city_data[city_key] = {
            'name': city_info['name'],
            'color': city_info['color'],
            'coords': (lat, lon),
            'graph': G_proj,
            'buildings': buildings_proj,
            'parcels': parcels_proj,
            'crs': ox.graph_to_gdfs(G_proj, nodes=False).crs
        }
        
        print(f"  ✓ Downloaded:")
        print(f"    - {G_proj.number_of_nodes()} nodes")
        print(f"    - {G_proj.number_of_edges()} edges")
        print(f"    - {len(buildings_proj)} buildings")
        print(f"    - {len(parcels_proj)} parcels")
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()

print(f"\n{'='*60}")
print(f"✓ Data acquisition complete for {len(city_data)} cities")
print(f"{'='*60}")

## 2.5 NEW: Base Map Visualization
Display the full urban context before analysis

In [ ]:
# Base maps showing roads + buildings (Option B: Vector data)
print("\n" + "="*60)
print("Creating base maps...")
print("="*60)

fig, axes = plt.subplots(1, 3, figsize=(24, 8), facecolor='white')

for idx, city_key in enumerate(city_data.keys()):
    ax = axes[idx]
    
    _, edges = ox.graph_to_gdfs(city_data[city_key]['graph'])
    buildings = city_data[city_key]['buildings']
    parcels = city_data[city_key]['parcels']
    
    # Plot parcels in light green (if available)
    if len(parcels) > 0:
        parcels.plot(ax=ax, color='#E8F5E9', edgecolor='#66BB6A', linewidth=0.5, alpha=0.3)
    
    # Plot buildings in gray
    buildings.plot(ax=ax, color='#BDBDBD', edgecolor='#424242', linewidth=0.3, alpha=0.8)
    
    # Plot roads in black
    edges.plot(ax=ax, color='#000000', linewidth=1.5, alpha=0.9)
    
    ax.set_title(
        f"{city_data[city_key]['name']}\n"
        f"{len(buildings)} buildings · {len(edges)} roads · {len(parcels)} parcels",
        fontsize=14, fontweight='bold', pad=15
    )
    ax.set_xlabel('Easting (m)', fontsize=11)
    ax.set_ylabel('Northing (m)', fontsize=11)
    ax.tick_params(labelsize=9)
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

plt.suptitle('Base Maps: Urban Context (500×500m)', fontsize=22, fontweight='bold', y=1.0)
plt.tight_layout()

# Save both formats
plt.savefig(VIZ_PNG_DIR / '00_base_maps.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.savefig(VIZ_SVG_DIR / '00_base_maps.svg', bbox_inches='tight', facecolor='white')
plt.show()

print("✓ Saved: 00_base_maps.png + 00_base_maps.svg")
print("  (Buildings in gray, roads in black, parcels in light green)")

## 3. Node Analysis (Centrality Metrics)

In [ ]:
def compute_node_metrics(G):
    """Compute centrality metrics for nodes"""
    print("  Computing node centrality...")
    
    G_undir = G.to_undirected()
    
    # Betweenness centrality
    print("    - Betweenness (distance)...")
    bc_dist = nx.betweenness_centrality(G_undir, weight='length', normalized=True)
    bc_info = nx.betweenness_centrality(G_undir, weight=None, normalized=True)
    
    # Closeness centrality
    print("    - Closeness...")
    closeness = nx.closeness_centrality(G_undir, distance='length')
    
    # Reach centrality
    print("    - Reach (200m, 300m)...")
    reach_200 = {}
    reach_300 = {}
    for node in G_undir.nodes():
        reach_200[node] = len(nx.single_source_dijkstra_path_length(G_undir, node, cutoff=200, weight='length'))
        reach_300[node] = len(nx.single_source_dijkstra_path_length(G_undir, node, cutoff=300, weight='length'))
    
    degree = dict(G_undir.degree())
    
    nodes, _ = ox.graph_to_gdfs(G)
    nodes['bc_distance'] = nodes.index.map(bc_dist)
    nodes['bc_information'] = nodes.index.map(bc_info)
    nodes['closeness'] = nodes.index.map(closeness)
    nodes['reach_200m'] = nodes.index.map(reach_200)
    nodes['reach_300m'] = nodes.index.map(reach_300)
    nodes['degree'] = nodes.index.map(degree)
    
    print("  ✓ Node metrics computed")
    return nodes

for city_key in city_data.keys():
    print(f"\n{city_data[city_key]['name']}:")
    city_data[city_key]['nodes'] = compute_node_metrics(city_data[city_key]['graph'])
    city_data[city_key]['nodes'].to_file(GEOJSON_DIR / f"{city_key}_nodes.geojson", driver='GeoJSON')
    print(f"  ✓ Saved to {city_key}_nodes.geojson")

## 4. Edge Analysis

In [ ]:
def compute_edge_metrics(G):
    """Compute edge metrics"""
    print("  Computing edge metrics...")
    
    G_undir = G.to_undirected()
    
    print("    - Edge betweenness...")
    edge_bc = nx.edge_betweenness_centrality(G_undir, weight='length', normalized=True)
    
    # Dual graph for angular analysis
    print("    - Angular betweenness...")
    dual_G = nx.Graph()
    edge_to_node = {}
    for i, (u, v, k) in enumerate(G_undir.edges(keys=True)):
        edge_to_node[(u, v, k)] = i
        dual_G.add_node(i, primal_edge=(u, v, k))
    
    for node in G_undir.nodes():
        incident_edges = list(G_undir.edges(node, keys=True))
        for i in range(len(incident_edges)):
            for j in range(i+1, len(incident_edges)):
                e1, e2 = incident_edges[i], incident_edges[j]
                e1_norm = tuple(sorted([e1[0], e1[1]])) + (e1[2],)
                e2_norm = tuple(sorted([e2[0], e2[1]])) + (e2[2],)
                if e1_norm in edge_to_node and e2_norm in edge_to_node:
                    dual_G.add_edge(edge_to_node[e1_norm], edge_to_node[e2_norm])
    
    dual_bc = nx.betweenness_centrality(dual_G, weight=None, normalized=True) if dual_G.number_of_edges() > 0 else {}
    angular_bc = {}
    for dual_node, bc_val in dual_bc.items():
        primal_edge = dual_G.nodes[dual_node].get('primal_edge')
        if primal_edge:
            angular_bc[primal_edge] = bc_val
    
    _, edges = ox.graph_to_gdfs(G)
    edges['edge_bc'] = edges.index.map(lambda x: edge_bc.get((x[0], x[1]), 0))
    edges['angular_bc'] = edges.index.map(lambda x: angular_bc.get(x, 0))
    
    print("  ✓ Edge metrics computed")
    return edges

for city_key in city_data.keys():
    print(f"\n{city_data[city_key]['name']}:")
    city_data[city_key]['edges'] = compute_edge_metrics(city_data[city_key]['graph'])
    city_data[city_key]['edges'].to_file(GEOJSON_DIR / f"{city_key}_edges.geojson", driver='GeoJSON')
    print(f"  ✓ Saved to {city_key}_edges.geojson")

## 5. Parcel Analysis
Process building parcels (landuse boundaries)

In [ ]:
# Process parcels and compute metrics
for city_key in city_data.keys():
    print(f"\n{city_data[city_key]['name']}:")
    parcels = city_data[city_key]['parcels']
    
    if len(parcels) > 0:
        print("  Processing parcels...")
        parcels['area'] = parcels.geometry.area
        parcels['perimeter'] = parcels.geometry.length
        parcels['compactness'] = (4 * np.pi * parcels['area']) / (parcels['perimeter'] ** 2)
        
        # Filter by size
        parcels_filtered = parcels[
            (parcels['area'] >= MIN_PARCEL_AREA) & 
            (parcels['area'] <= MAX_PARCEL_AREA)
        ].copy()
        
        # Compute aspect ratio
        aspect_ratios = []
        for geom in parcels_filtered.geometry:
            try:
                mbr = geom.minimum_rotated_rectangle
                coords = list(mbr.exterior.coords)
                side1 = Point(coords[0]).distance(Point(coords[1]))
                side2 = Point(coords[1]).distance(Point(coords[2]))
                aspect = max(side1, side2) / min(side1, side2) if min(side1, side2) > 0 else 1.0
                aspect_ratios.append(aspect)
            except:
                aspect_ratios.append(1.0)
        
        parcels_filtered['aspect_ratio'] = aspect_ratios
        parcels_filtered['parcel_id'] = [f"parcel_{i:03d}" for i in range(len(parcels_filtered))]
        
        city_data[city_key]['parcels_processed'] = parcels_filtered
        
        # Save
        parcels_filtered.to_file(GEOJSON_DIR / f"{city_key}_parcels.geojson", driver='GeoJSON')
        print(f"  ✓ Processed {len(parcels_filtered)} parcels (filtered from {len(parcels)})")
        print(f"  ✓ Saved to {city_key}_parcels.geojson")
    else:
        print("  ⚠ No parcels available")
        city_data[city_key]['parcels_processed'] = gpd.GeoDataFrame()

## 6. District Analysis

In [ ]:
# NetworkX has built-in Louvain community detection (since v2.5+)
# No external packages needed
print("✓ Using NetworkX built-in Louvain community detection")

In [ ]:
def detect_districts(G, method='distance'):
    """Detect districts using community detection"""
    print(f"    - {method}...")
    try:
        G_undir = G.to_undirected()
        G_simple = nx.Graph()
        for u, v, data in G_undir.edges(data=True):
            if not G_simple.has_edge(u, v):
                G_simple.add_edge(u, v, **data)
        
        # Use NetworkX built-in Louvain
        if method == 'distance':
            communities = nx.algorithms.community.louvain_communities(G_simple, weight='length')
        else:
            communities = nx.algorithms.community.louvain_communities(G_simple, weight=None)
        
        # Convert from list of sets to node->community_id dict
        partition = {}
        for comm_id, community in enumerate(communities):
            for node in community:
                partition[node] = comm_id
        
        return partition
    except Exception as e:
        print(f"      ✗ Error: {e}")
        return {node: 0 for node in G.nodes()}

for city_key in city_data.keys():
    print(f"\n{city_data[city_key]['name']}:")
    G = city_data[city_key]['graph']
    nodes = city_data[city_key]['nodes']
    
    partitions = {}
    for method in ['distance', 'angular', 'topological']:
        partition = detect_districts(G, method=method)
        partitions[method] = partition
        
        nodes_districts = nodes.copy()
        nodes_districts['district'] = nodes_districts.index.map(partition)
        nodes_districts.to_file(GEOJSON_DIR / f"{city_key}_districts_{method}.geojson", driver='GeoJSON')
        
        print(f"      {method}: {len(set(partition.values()))} districts")
    
    city_data[city_key]['partitions'] = partitions
    print(f"  ✓ District detection complete")

## 7. Landmark Analysis

In [ ]:
def safe_normalize(series, default=0.5):
    """Safely normalize, handling NaN and min==max"""
    min_val, max_val = series.min(), series.max()
    if pd.isna(min_val) or pd.isna(max_val) or min_val == max_val:
        return pd.Series([default] * len(series), index=series.index)
    return ((series - min_val) / (max_val - min_val)).fillna(default)

def compute_landmark_scores(buildings_gdf, edges_gdf):
    """Compute landmark scores"""
    print("  Computing landmark scores...")
    buildings = buildings_gdf.copy()
    
    # Structural score
    buildings['area'] = buildings.geometry.area
    buildings['s_area'] = safe_normalize(buildings['area'])
    
    street_union = unary_union(edges_gdf.geometry)
    buildings['dist_to_street'] = buildings.geometry.apply(lambda g: g.distance(street_union))
    max_dist = buildings['dist_to_street'].max()
    buildings['s_visibility'] = (1 - buildings['dist_to_street'] / max_dist) if max_dist > 0 else 0.5
    buildings['s_visibility'] = buildings['s_visibility'].fillna(0.5)
    buildings['structural_score'] = (0.6 * buildings['s_area'] + 0.4 * buildings['s_visibility']).fillna(0.5)
    
    # Other scores
    buildings['visual_score'] = 0.5
    buildings['cultural_score'] = 0.0
    buildings['pragmatic_score'] = 0.0
    buildings['global_score'] = (0.4 * buildings['structural_score'] + 0.2 * buildings['visual_score'] + 
                                   0.2 * buildings['cultural_score'] + 0.2 * buildings['pragmatic_score']).fillna(0.5)
    
    # Geometry metrics
    aspect_ratios = []
    for geom in buildings.geometry:
        try:
            mbr = geom.minimum_rotated_rectangle
            coords = list(mbr.exterior.coords)
            side1 = Point(coords[0]).distance(Point(coords[1]))
            side2 = Point(coords[1]).distance(Point(coords[2]))
            aspect_ratios.append(max(side1, side2) / min(side1, side2) if min(side1, side2) > 0 else 1.0)
        except:
            aspect_ratios.append(1.0)
    buildings['aspect_ratio'] = aspect_ratios
    buildings['setback_dist'] = buildings['dist_to_street']
    
    print("  ✓ Landmark scores computed")
    return buildings

for city_key in city_data.keys():
    print(f"\n{city_data[city_key]['name']}:")
    buildings_scored = compute_landmark_scores(city_data[city_key]['buildings'], city_data[city_key]['edges'])
    city_data[city_key]['buildings_scored'] = buildings_scored
    
    cols = ['geometry', 'area', 'structural_score', 'global_score', 'aspect_ratio', 'setback_dist']
    buildings_scored[cols].to_file(GEOJSON_DIR / f"{city_key}_buildings.geojson", driver='GeoJSON')
    print(f"  ✓ Saved to {city_key}_buildings.geojson")

## 8. Building Footprint Library
Extract diverse individual building footprints (not blocks!)

In [ ]:
def extract_building_footprint_library(buildings_gdf, city_key, target_count=35):
    """Extract diverse building footprints"""
    print(f"  Extracting {target_count} footprints...")
    
    if len(buildings_gdf) == 0:
        return []
    
    # Sort by area for diversity
    buildings = buildings_gdf.copy().sort_values('area')
    
    if len(buildings) <= target_count:
        selected = buildings
    else:
        indices = np.linspace(0, len(buildings)-1, target_count, dtype=int)
        selected = buildings.iloc[indices]
    
    library = []
    for idx, (_, row) in enumerate(selected.iterrows()):
        geom = row.geometry
        centroid = geom.centroid
        
        # Translate to origin
        translated = translate(geom, xoff=-centroid.x, yoff=-centroid.y)
        
        library.append({
            'footprint_id': f"{city_key}_building_{idx:03d}",
            'city': city_key,
            'area': float(row['area']),
            'aspect_ratio': float(row.get('aspect_ratio', 1.0)),
            'structural_score': float(row.get('structural_score', 0.5)),
            'geometry': {
                'type': 'Polygon',
                'coordinates': [list(translated.exterior.coords)]
            }
        })
    
    print(f"  ✓ Extracted {len(library)} footprints")
    return library

# Extract for all cities
all_footprints = []
for city_key in city_data.keys():
    print(f"\n{city_data[city_key]['name']}:")
    footprints = extract_building_footprint_library(
        city_data[city_key]['buildings_scored'],
        city_key,
        FOOTPRINTS_PER_CITY
    )
    all_footprints.extend(footprints)
    city_data[city_key]['footprint_library'] = footprints

print(f"\n{'='*60}")
print(f"✓ Total footprint library: {len(all_footprints)} buildings")
print(f"{'='*60}")

# Save
if len(all_footprints) > 0:
    with open(METRICS_DIR / 'building_footprint_library.json', 'w') as f:
        json.dump(all_footprints, f, indent=2)
    print(f"✓ Saved to building_footprint_library.json")

## 9. Metrics Aggregation

In [ ]:
def compute_distribution(values, bins=20):
    if len(values) == 0:
        return {'bins': [], 'counts': [], 'mean': 0, 'median': 0, 'std': 0, 'min': 0, 'max': 0}
    hist, bin_edges = np.histogram(values, bins=bins)
    return {
        'bins': bin_edges.tolist(),
        'counts': hist.tolist(),
        'mean': float(np.mean(values)),
        'median': float(np.median(values)),
        'std': float(np.std(values)),
        'min': float(np.min(values)),
        'max': float(np.max(values))
    }

urban_metrics = {}
for city_key in city_data.keys():
    print(f"\nAggregating: {city_data[city_key]['name']}...")
    
    nodes = city_data[city_key]['nodes']
    edges = city_data[city_key]['edges']
    parcels = city_data[city_key]['parcels_processed']
    buildings = city_data[city_key]['buildings_scored']
    partitions = city_data[city_key]['partitions']
    
    urban_metrics[city_key] = {
        'name': city_data[city_key]['name'],
        'nodes': {
            'total_count': len(nodes),
            'avg_degree': float(nodes['degree'].mean()),
            'degree_distribution': nodes['degree'].value_counts().to_dict()
        },
        'edges': {
            'total_count': len(edges),
            'total_length_km': float(edges['length'].sum() / 1000),
            'density_km_per_km2': float((edges['length'].sum() / 1000) / 0.25),
            'segment_length_distribution': compute_distribution(edges['length'].values)
        },
        'parcels': {
            'total_count': len(parcels),
            'area_distribution': compute_distribution(parcels['area'].values) if len(parcels) > 0 else {}
        },
        'buildings': {
            'total_count': len(buildings),
            'area_distribution': compute_distribution(buildings['area'].values),
            'aspect_ratio_distribution': compute_distribution(buildings['aspect_ratio'].values)
        },
        'districts': {
            'count_distance': len(set(partitions['distance'].values())),
            'count_angular': len(set(partitions['angular'].values())),
            'count_topological': len(set(partitions['topological'].values()))
        }
    }

with open(METRICS_DIR / 'urban_metrics.json', 'w') as f:
    json.dump({'urban_metrics': urban_metrics}, f, indent=2)

print(f"\n{'='*60}")
print(f"✓ Saved to urban_metrics.json")
print(f"{'='*60}")

## 10. Metrics Summary Table

In [ ]:
# Create summary table
print("\n" + "="*80)
print("📊 METRICS SUMMARY TABLE")
print("="*80)

table_data = []
metrics = [
    ('Nodes', lambda m: m['nodes']['total_count']),
    ('Edges', lambda m: m['edges']['total_count']),
    ('Street Length (km)', lambda m: m['edges']['total_length_km']),
    ('Street Density (km/km²)', lambda m: m['edges']['density_km_per_km2']),
    ('Avg Segment (m)', lambda m: m['edges']['segment_length_distribution']['mean']),
    ('Parcels', lambda m: m['parcels']['total_count']),
    ('Buildings', lambda m: m['buildings']['total_count']),
    ('Avg Building Area (m²)', lambda m: m['buildings']['area_distribution']['mean']),
    ('Districts (distance)', lambda m: m['districts']['count_distance'])
]

for metric_name, func in metrics:
    row = {'Metric': metric_name}
    for city_key in city_data.keys():
        try:
            row[city_data[city_key]['name']] = f"{func(urban_metrics[city_key]):.1f}"
        except:
            row[city_data[city_key]['name']] = 'N/A'
    table_data.append(row)

df = pd.DataFrame(table_data).set_index('Metric')
print("\n" + df.to_string())

df.to_csv(METRICS_DIR / 'metrics_summary.csv')
print(f"\n✓ Saved to metrics_summary.csv")
print("="*80)

## 11. Visualizations (All SVG + PNG)

In [ ]:
# Betweenness comparison
fig, axes = plt.subplots(1, 3, figsize=(24, 8), facecolor='#1a1a1a')

for idx, city_key in enumerate(city_data.keys()):
    ax = axes[idx]
    edges = city_data[city_key]['edges']
    edges.plot(ax=ax, column='angular_bc', cmap='YlOrRd', linewidth=2, legend=False)
    ax.set_title(city_data[city_key]['name'], fontsize=20, color='white')
    ax.axis('off')
    ax.set_facecolor('#1a1a1a')

plt.suptitle('Angular Betweenness: Urban Movement', fontsize=24, color='white')
plt.tight_layout()
plt.savefig(VIZ_PNG_DIR / 'betweenness_comparison.png', dpi=300, facecolor='#1a1a1a', bbox_inches='tight')
plt.savefig(VIZ_SVG_DIR / 'betweenness_comparison.svg', facecolor='#1a1a1a', bbox_inches='tight')
plt.show()
print("✓ Saved: betweenness_comparison (PNG + SVG)")

In [ ]:
# Footprint library visualization
if len(all_footprints) > 0:
    fig, axes = plt.subplots(3, 4, figsize=(20, 15), facecolor='white')
    axes = axes.flatten()
    
    selected = []
    for city_key in city_data.keys():
        lib = city_data[city_key]['footprint_library']
        if len(lib) >= 4:
            selected.extend([lib[0], lib[len(lib)//3], lib[2*len(lib)//3], lib[-1]])
    
    for idx, fp in enumerate(selected[:12]):
        ax = axes[idx]
        poly = Polygon(fp['geometry']['coordinates'][0])
        x, y = poly.exterior.xy
        ax.fill(x, y, color='black')
        ax.set_title(f"{fp['city'].upper()}\n{fp['area']:.0f} m² | AR: {fp['aspect_ratio']:.1f}", fontsize=10)
        ax.set_aspect('equal')
        ax.axis('off')
    
    for idx in range(len(selected), 12):
        axes[idx].axis('off')
    
    plt.suptitle('Building Footprint Library', fontsize=24)
    plt.tight_layout()
    plt.savefig(VIZ_PNG_DIR / 'footprint_library.png', dpi=300, bbox_inches='tight')
    plt.savefig(VIZ_SVG_DIR / 'footprint_library.svg', bbox_inches='tight')
    plt.show()
    print("✓ Saved: footprint_library (PNG + SVG)")

In [ ]:
# Comparative histograms
fig, axes = plt.subplots(2, 2, figsize=(18, 14), facecolor='white')

distributions = [
    ('edges', 'segment_length_distribution', 'Street Segment Length', 'Length (m)'),
    ('parcels', 'area_distribution', 'Parcel Area', 'Area (m²)'),
    ('buildings', 'area_distribution', 'Building Area', 'Area (m²)'),
    ('buildings', 'aspect_ratio_distribution', 'Building Aspect Ratio', 'Ratio')
]

for idx, (cat, key, title, xlabel) in enumerate(distributions):
    ax = axes[idx // 2, idx % 2]
    for city_key in city_data.keys():
        metric = urban_metrics[city_key][cat].get(key, {})
        if 'bins' in metric and len(metric['bins']) > 1:
            centers = [(metric['bins'][i] + metric['bins'][i+1])/2 for i in range(len(metric['bins'])-1)]
            ax.plot(centers, metric['counts'], 
                   label=f"{city_data[city_key]['name']} (μ={metric['mean']:.1f})",
                   color=city_data[city_key]['color'], linewidth=2.5, alpha=0.8)
    ax.set_xlabel(xlabel, fontsize=13, fontweight='bold')
    ax.set_ylabel('Frequency', fontsize=13, fontweight='bold')
    ax.set_title(title, fontsize=15, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)

plt.suptitle('Comparative Distributions', fontsize=20, fontweight='bold')
plt.tight_layout()
plt.savefig(VIZ_PNG_DIR / 'comparative_histograms.png', dpi=300, bbox_inches='tight')
plt.savefig(VIZ_SVG_DIR / 'comparative_histograms.svg', bbox_inches='tight')
plt.show()
print("✓ Saved: comparative_histograms (PNG + SVG)")

## 12. Final Summary

In [ ]:
print("\n" + "="*80)
print("✓ STEP 1 COMPLETE: URBAN ANALYSIS")
print("="*80)

print("\n📁 OUTPUTS:")
print(f"  GeoJSON: {len(list(GEOJSON_DIR.glob('*.geojson')))} files")
print(f"  PNG: {len(list(VIZ_PNG_DIR.glob('*.png')))} files")
print(f"  SVG: {len(list(VIZ_SVG_DIR.glob('*.svg')))} files")
print(f"  Metrics: {len(list(METRICS_DIR.glob('*')))} files")

print("\n📊 RESULTS:")
for city_key in city_data.keys():
    m = urban_metrics[city_key]
    print(f"  {m['name']}: {m['buildings']['total_count']} buildings, "
          f"{m['parcels']['total_count']} parcels, "
          f"{len(city_data[city_key]['footprint_library'])} in library")

print(f"\n  TOTAL FOOTPRINT LIBRARY: {len(all_footprints)} buildings")

print("\n✓ All visualizations: PNG + SVG")
print("✓ Building footprints: Individual shapes (not blocks)")
print("✓ Parcels: Landuse boundaries from OSM")
print("="*80)